In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
import random
import os
import torch
import sys
sys.path.append('../..')
from modules.many_features import utils, constants
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

C:\Users\User\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
torch.manual_seed(SEED)
torch.use_deterministic_algorithms(True)

#### The Data

In [3]:
#df = pd.read_csv('../../data/anemia_synth_dataset_some_nans_unspecified_more_feats.csv')
#df = pd.read_csv('../../data/more_feats_0.2.csv')
#df= pd.read_csv('../../data/more_features/more_feats_new_labels_0.1.csv')
#df =pd.read_csv('../../data/more_features/more_feats_new_labels_0.1_noisy_0.6.csv')
df = pd.read_csv('../../data/more_features//more_feats_correlated_noisy_4.csv')
#df = utils.balance_dataset(df, 8000)
df = df.fillna(-1)
df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,8.725667,-1.000000,2.344526,-1.000000,-1.000000,96.678418,-1.000000,2.707637,1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,26.177001,-1.000000,Hemolytic anemia
1,12.430232,231.691743,5.143660,5.339158,484.590922,87.353532,-1.000000,4.268940,0,-1.000000,-1.000000,-1.000000,8.718062,-1.000000,-1.000000,37.290695,-1.000000,No anemia
2,6.737760,360.843434,-1.000000,5.481140,421.177091,75.086568,82.986344,2.691997,0,0.458698,19.596767,69.527433,-1.000000,23.027617,-1.000000,20.213279,19.703433,Anemia of chronic disease
3,12.313034,-1.000000,2.765581,0.000000,-1.000000,99.494524,-1.000000,3.712677,1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,36.939103,-1.000000,Unspecified anemia
4,6.758140,-1.000000,1.696525,5.648115,-1.000000,86.643128,199.741346,2.339992,0,0.887158,80.614718,109.197814,18.770122,7.668335,63.159054,20.274421,-1.000000,Aplastic anemia


In [4]:
df.isna().sum()

hemoglobin               0
ferritin                 0
ret_count                0
segmented_neutrophils    0
tibc                     0
mcv                      0
serum_iron               0
rbc                      0
gender                   0
creatinine               0
cholestrol               0
copper                   0
ethanol                  0
folate                   0
glucose                  0
hematocrit               0
tsat                     0
label                    0
dtype: int64

In [5]:
utils.get_dt_performance(df)

(0.7016428571428571,
 0.7005761070620158,
 0.8266962007922731,
 datetime.timedelta(microseconds=4772))

In [6]:
df.label.value_counts()

No anemia                               16000
Anemia of chronic disease                8816
Iron deficiency anemia                   8309
Aplastic anemia                          8157
Unspecified anemia                       8132
Vitamin B12/Folate deficiency anemia     8124
Hemolytic anemia                         8057
Inconclusive diagnosis                   4405
Name: label, dtype: int64

In [7]:
class_dict = constants.CLASS_DICT
df['label'] = df['label'].replace(class_dict)
X = df.iloc[:, 0:-1]
y = df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=SEED)
X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((56000, 17), (14000, 17), (56000,), (14000,))

In [8]:
action_list = list(class_dict.keys()) + [col  for col in df.columns if col!='label']
action_list

['No anemia',
 'Vitamin B12/Folate deficiency anemia',
 'Unspecified anemia',
 'Anemia of chronic disease',
 'Iron deficiency anemia',
 'Hemolytic anemia',
 'Aplastic anemia',
 'Inconclusive diagnosis',
 'hemoglobin',
 'ferritin',
 'ret_count',
 'segmented_neutrophils',
 'tibc',
 'mcv',
 'serum_iron',
 'rbc',
 'gender',
 'creatinine',
 'cholestrol',
 'copper',
 'ethanol',
 'folate',
 'glucose',
 'hematocrit',
 'tsat']

In [9]:
len(action_list)

25

In [10]:
df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,8.725667,-1.000000,2.344526,-1.000000,-1.000000,96.678418,-1.000000,2.707637,1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,26.177001,-1.000000,5
1,12.430232,231.691743,5.143660,5.339158,484.590922,87.353532,-1.000000,4.268940,0,-1.000000,-1.000000,-1.000000,8.718062,-1.000000,-1.000000,37.290695,-1.000000,0
2,6.737760,360.843434,-1.000000,5.481140,421.177091,75.086568,82.986344,2.691997,0,0.458698,19.596767,69.527433,-1.000000,23.027617,-1.000000,20.213279,19.703433,3
3,12.313034,-1.000000,2.765581,0.000000,-1.000000,99.494524,-1.000000,3.712677,1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,36.939103,-1.000000,2
4,6.758140,-1.000000,1.696525,5.648115,-1.000000,86.643128,199.741346,2.339992,0,0.887158,80.614718,109.197814,18.770122,7.668335,63.159054,20.274421,-1.000000,6


#### Training 

In [13]:
# %%time
# timesteps = int(2e6)
# dqn_model = utils.stable_dqn3(X_train, y_train, timesteps, True, f'../../models/many_features/stable_dqn3_{timesteps}')
# test_df = utils.evaluate_dqn(dqn_model, X_test, y_test)
# test_df.head()

In [ ]:
#for steps in [int(6e6), int(6.5e6), int(7e6), int(7.5e6), int(8e6), int(8.5e6), int(9e6)]:
for steps in [int(9.5e6), int(10.5e6), int(11.5e6), int(12.5e6), int(13.5e6)]:
    #start_time = datetime.now()
    dqn_model = utils.stable_dqn3(X_train, y_train, steps, True, 
                                  f'../../models/many_features/0.1/with_correlated_fts/dqn3_by_type_noisy_4_{steps}')
    #end_time = datetime.now()
    #print(f'The duration for {steps} steps is {end_time-start_time}')

using stable baselines 3
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.28     |
|    ep_rew_mean      | -0.84    |
|    exploration_rate | 0.705    |
|    success_rate     | 0.1      |
| time/               |          |
|    episodes         | 100000   |
|    fps              | 623      |
|    time_elapsed     | 473      |
|    total_timesteps  | 295184   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 7.22     |
|    n_updates        | 61295    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.83     |
|    ep_rew_mean      | -0.86    |
|    exploration_rate | 0.346    |
|    success_rate     | 0.14     |
| time/               |          |
|    episodes         | 200000   |
|    fps              | 555      |
|    t

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 4.42     |
|    ep_rew_mean      | -0.22    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.4      |
| time/               |          |
|    episodes         | 1600000  |
|    fps              | 570      |
|    time_elapsed     | 11588    |
|    total_timesteps  | 6613768  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.149    |
|    n_updates        | 1640941  |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.75     |
|    ep_rew_mean      | -0.24    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.4      |
| time/               |          |
|    episodes         | 1700000  |
|    fps              | 571      |
|    time_elapsed     | 12306    |
|    total_timesteps  | 7032294  |
| train/              |          |
|    learning_rate  

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 4.18     |
|    ep_rew_mean      | -0.48    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.28     |
| time/               |          |
|    episodes         | 900000   |
|    fps              | 632      |
|    time_elapsed     | 5730     |
|    total_timesteps  | 3622152  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0272   |
|    n_updates        | 893037   |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 4.23     |
|    ep_rew_mean      | -0.3     |
|    exploration_rate | 0.05     |
|    success_rate     | 0.4      |
| time/               |          |
|    episodes         | 1000000  |
|    fps              | 623      |
|    time_elapsed     | 6476     |
|    total_timesteps  | 4037795  |
| train/              |          |
|    learning_rate  

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 4.12     |
|    ep_rew_mean      | 0.04     |
|    exploration_rate | 0.05     |
|    success_rate     | 0.56     |
| time/               |          |
|    episodes         | 2400000  |
|    fps              | 504      |
|    time_elapsed     | 19256    |
|    total_timesteps  | 9716273  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.159    |
|    n_updates        | 2416568  |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 4.61     |
|    ep_rew_mean      | 0.08     |
|    exploration_rate | 0.05     |
|    success_rate     | 0.59     |
| time/               |          |
|    episodes         | 2500000  |
|    fps              | 505      |
|    time_elapsed     | 20060    |
|    total_timesteps  | 10133800 |
| train/              |          |
|    learning_rate  

#### Testing

In [50]:
# training_env = utils.create_env(X_train, y_train)
# dqn_model = utils.load_dqn3('../../models/many_features/0.1/with_correlated_fts/dqn3_by_type_noisy_4_15000000', training_env)
# test_df = utils.evaluate_dqn(dqn_model, X_test, y_test)
# test_df.head()

Using stable baselines 3
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Count: 2800
Count: 5600
Count: 8400
Count: 11200
Count: 14000
Testing done.....


,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred
0,6.0,0.0,1.0,1.0,0.0,"[hemoglobin, rbc, mcv, tibc, gender, Iron defi...",4.0,4.0
1,5.0,1.0,1.0,1.0,0.0,"[hemoglobin, rbc, mcv, tibc, Anemia of chronic...",3.0,3.0
2,5.0,2.0,1.0,1.0,0.0,"[hemoglobin, rbc, mcv, tibc, Anemia of chronic...",3.0,3.0
3,6.0,3.0,1.0,1.0,0.0,"[hemoglobin, gender, rbc, mcv, tibc, Iron defi...",4.0,4.0
4,4.0,4.0,0.0,-1.0,0.0,"[hemoglobin, rbc, ret_count, Hemolytic anemia]",2.0,5.0


In [51]:
success_rate, success_df = utils.success_rate(test_df)
success_rate

79.11428571428571

In [52]:
avg_length, avg_return = utils.get_avg_length_reward(test_df)
avg_length, avg_return

(4.2282142857142855, 0.45685714285714285)

In [53]:
acc, f1, roc_auc = utils.test(test_df['y_actual'], test_df['y_pred'])
acc, f1, roc_auc

(0.7911428571428571, 0.7914943705040569, 0.8979100156731731)

In [54]:
test_df.y_pred.unique()

array([4., 3., 5., 1., 0., 6., 7., 2.])

In [19]:
test_df[test_df.y_pred==4]

,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred
0,4.0,0.0,1.0,1.0,0.0,"[hemoglobin, rbc, ferritin, Iron deficiency an...",4.0,4.0
1,4.0,1.0,0.0,-1.0,0.0,"[hemoglobin, rbc, ferritin, Iron deficiency an...",3.0,4.0
8,5.0,8.0,1.0,1.0,0.0,"[hemoglobin, gender, rbc, ferritin, Iron defic...",4.0,4.0
11,4.0,11.0,1.0,1.0,0.0,"[hemoglobin, rbc, ferritin, Iron deficiency an...",4.0,4.0
36,4.0,36.0,0.0,-1.0,0.0,"[hemoglobin, rbc, ferritin, Iron deficiency an...",3.0,4.0
...,...,...,...,...,...,...,...,...
13966,4.0,13966.0,1.0,1.0,0.0,"[hemoglobin, rbc, ferritin, Iron deficiency an...",4.0,4.0
13970,4.0,13970.0,0.0,-1.0,0.0,"[hemoglobin, rbc, ferritin, Iron deficiency an...",7.0,4.0
13976,4.0,13976.0,1.0,1.0,0.0,"[hemoglobin, rbc, ferritin, Iron deficiency an...",4.0,4.0
13983,5.0,13983.0,1.0,1.0,0.0,"[hemoglobin, gender, rbc, ferritin, Iron defic...",4.0,4.0


#### Saving files

In [75]:
# test_df.to_csv(f'../../test_dfs/many_features/0.1/correlated/test_df3_9000000.csv', index=False)
# success_df.to_csv(f'../../test_dfs/many_features/0.1/correlated/success_df3_9000000.csv', index=False)

#### Confusion matrix and classification report

In [ ]:
test_df = pd.read_csv('../../test_dfs/many_features/0.1/test_df3_6500000.csv')
test_df.head()

In [ ]:
utils.plot_classification_report(test_df['y_actual'], test_df['y_pred'])

In [ ]:
# def plot_confusion_matrix(y_actual, y_pred, save=False, filename=False):
#     from sklearn.metrics import confusion_matrix
#     cm = confusion_matrix(y_actual, y_pred)
#     cm_df = pd.DataFrame(cm, index = [0, 1, 2, 3, 4, 5, 6], columns = [0, 1, 2, 3, 4, 5, 6], dtype='object')
#     #cm_df = pd.DataFrame(cm, index = constants.CLASS_DICT.keys(), columns = constants.CLASS_DICT.keys())
#     plt.figure(figsize=(8, 6))
#     sns.heatmap(cm_df, annot=True)
#     plt.title('Confusion Matrix')
#     plt.ylabel('Actual Anemia')
#     plt.xlabel('Predicted Anemia')
#     plt.tight_layout()
#     if save:
#         plt.savefig(filename)
#     plt.show()
#     plt.close()

In [ ]:
utils.plot_confusion_matrix(test_df['y_actual'], test_df['y_pred'])